# Optiver Trading At The Close

[Competition](https://www.kaggle.com/competitions/optiver-trading-at-the-close)

[GitHub Repository](https://github.com/cvaisnor/DNN-Final-Project)

[Google Slides Presentation](https://docs.google.com/presentation/d/1Xc5F1_NveFi1il3GqHej2aqVmklR_jZU4kx6ZJhiDEM/edit?usp=sharing)

# General Imports

In [135]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

kaggle_environment = False # True if running on Kaggle, don't forget to add the dataset!

if kaggle_environment:
    data_path = '/kaggle/input/'
else:
    data_path = 'kaggle/input/'

# disable PkgResourcesDeprecationWarning
import warnings
warnings.filterwarnings('ignore')

# Setting File Structure

In [136]:
for dirname, _, filenames in os.walk(data_path):
    for filename in filenames:
        print(os.path.join(dirname, filename))

kaggle/input/optiver-trading-at-the-close/public_timeseries_testing_util.py
kaggle/input/optiver-trading-at-the-close/.gitkeep
kaggle/input/optiver-trading-at-the-close/train.csv
kaggle/input/optiver-trading-at-the-close/optiver2023/__init__.py
kaggle/input/optiver-trading-at-the-close/optiver2023/competition.cpython-310-x86_64-linux-gnu.so
kaggle/input/optiver-trading-at-the-close/optiver2023/__pycache__/__init__.cpython-310.pyc
kaggle/input/optiver-trading-at-the-close/example_test_files/test.csv
kaggle/input/optiver-trading-at-the-close/example_test_files/revealed_targets.csv
kaggle/input/optiver-trading-at-the-close/example_test_files/sample_submission.csv


# Loading Data

In [137]:
train = pd.read_csv(data_path + 'optiver-trading-at-the-close/train.csv')
revealed_targets = pd.read_csv(data_path + 'optiver-trading-at-the-close/example_test_files/revealed_targets.csv')
test = pd.read_csv(data_path + 'optiver-trading-at-the-close/example_test_files/test.csv')
sample_submission = pd.read_csv(data_path + 'optiver-trading-at-the-close/example_test_files/sample_submission.csv')

# Feature Engineering Section

In [138]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

class CustomIterativeImputer(BaseEstimator, TransformerMixin):
    def __init__(self, columns, max_iter=10, random_state=0):
        self.columns = columns
        self.max_iter = max_iter
        self.random_state = random_state
        self.imputer = None

    def fit(self, X, y=None):
        # Initialize the IterativeImputer
        self.imputer = IterativeImputer(max_iter=self.max_iter, random_state=self.random_state)
        self.imputer.fit(X[self.columns])
        return self

    def transform(self, X):
        # Check is fit had been called
        if self.imputer is None:
            raise RuntimeError("You must call fit before calling transform")
        
        # Fit the imputer and transform the data
        X[self.columns] = self.imputer.transform(X[self.columns])
        return X

# Create the pipeline with the custom imputer
pipeline = Pipeline([
    ('iterative_imputer', CustomIterativeImputer(columns=['far_price', 'near_price'])),
    # Add other steps here if necessary
])

In [139]:
# Use the pipeline on the training data
df_processed = pipeline.fit_transform(train)

# drop all rows with NaNs (500-ish rows after imputation)
df_processed = df_processed.dropna()

# drop row_id column
df_processed = df_processed.drop(columns=['row_id'])

In [140]:
def add_datetime_feature(df):
    """
    Add datetime features to a DataFrame.

    Parameters:
    df (pandas.DataFrame): DataFrame to process.

    Returns:
    pandas.DataFrame: DataFrame with added datetime feature.
    """
    df = df.copy()
    start_time = pd.to_timedelta('9:30:00')
    # Convert date_id to actual dates starting from 2020-01-01
    df['date'] = pd.to_datetime('2020-01-01') + pd.to_timedelta(df['date_id'], unit='D')
    df['time'] = pd.to_timedelta(df['time_id'], unit='s')
    # Add the trading start time to the time column
    df['time'] = df['time'] + start_time
    # Create long-form datetime timestamp
    df['timestamp'] = df['date'] + df['time']
    df.drop(['date', 'time', 'date_id'], axis=1, inplace=True)

    # add day
    # df['day'] = df['timestamp'].dt.day

    # Add unix time
    # df['unix_time'] = df['timestamp'].astype(np.int64) // 10**9

    # Add a time_idx (an sequence of consecutive integers that goes from min to max date)
    # df['time_idx'] = df['timestamp'].rank(method='dense').astype(int) - 1

    return df

In [141]:
df_with_datetime = add_datetime_feature(df_processed)

In [142]:
def generate_features(test):
    return test

# Model Development

In [162]:
import os
import warnings

warnings.filterwarnings("ignore")  # avoid printing out absolute paths

os.chdir("../../..")

In [163]:
import copy
from pathlib import Path
import warnings

import lightning.pytorch as pl
from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor
from lightning.pytorch.loggers import TensorBoardLogger
import numpy as np
import pandas as pd
import torch

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import MAE, SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

In [158]:
# imports for Temporal Fusion Transformer

import copy
from pathlib import Path
import warnings
import seaborn as sns
import matplotlib
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

import numpy as np
import pandas as pd
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
import torch
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer, NaNLabelEncoder
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss, MAE
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters
import random
import gc
import tensorflow as tf 

random.seed(30)
np.random.seed(30)
tf.random.set_seed(30)
torch.manual_seed(30)
torch.cuda.manual_seed(30)

In [144]:
# we want to predict 1 minute ahead of each stock_id
max_prediction_length = 60 # (seconds)

# max encoder length = number of seconds in 7 hours
max_encoder_length = 7 * 60 * 60 # (seconds)

In [145]:
# get min and max datetimes in the dataset
min_time = df_with_datetime['timestamp'].min()
max_time = df_with_datetime['timestamp'].max()

# training cutoff will be the last 20 percent of the data
cutoff = min_time + (max_time - min_time) * 0.8

In [146]:
# convert "stock_id" to string so it is categorical
df_with_datetime['stock_id'] = df_with_datetime['stock_id'].astype(str)

In [148]:
# creating the dataset
training = TimeSeriesDataSet(
    df_with_datetime[df_with_datetime['timestamp'] <= cutoff],
    time_idx="time_id",
    target="target",
    group_ids=["stock_id"],
    min_encoder_length=0,  # allow predictions without history
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=max_prediction_length,
    static_categoricals=["stock_id"],
    time_varying_unknown_reals=["target"],
    allow_missing_timesteps=True,
)

In [150]:
# TypeError: Addition/subtraction of integers and integer-arrays with Timestamp is no longer supported.  Instead of adding/subtracting `n`, use `n * obj.freq`

# create validation set (predict=True) which means to predict the last max_prediction_length points in time for each series
validation = TimeSeriesDataSet.from_dataset(training, df_with_datetime, predict=True, stop_randomization=True)

In [151]:
# creating dataloaders
batch_size = 128  # set this between 32 to 128
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=0)

In [160]:
PATIENCE = 30
MAX_EPOCHS = 120
LEARNING_RATE = 0.03
OPTUNA = False

early_stopping_callback = EarlyStopping(monitor="mae", min_delta=1e-4, patience=PATIENCE, verbose=False, mode="min")

lr_logger = LearningRateMonitor()  # log the learning rate

# trainer = pl.Trainer(
#     max_epochs=MAX_EPOCHS,
#     gpus=1,
#     accelerator="gpu",
#     enable_model_summary=True,
#     gradient_clip_val=0.1,
#     limit_train_batches=30,  # coment in for training, running valiation every 30 batches
#     fast_dev_run=True,  # comment in to check that networkor dataset has no serious bugs
#     callbacks=[lr_logger, early_stopping_callback],
# )

# configure network and trainer
pl.seed_everything(42)
trainer = pl.Trainer(
    accelerator="gpu",
    # clipping gradients is a hyperparameter and important to prevent divergance
    # of the gradient for recurrent neural networks
    gradient_clip_val=0.1,
)

tft = TemporalFusionTransformer.from_dataset(
    training,
    # not meaningful for finding the learning rate but otherwise very important
    learning_rate=0.03,
    hidden_size=8,  # most important hyperparameter apart from learning rate
    # number of attention heads. Set to up to 4 for large datasets
    attention_head_size=1,
    dropout=0.1,  # between 0.1 and 0.3 are good values
    hidden_continuous_size=8,  # set to <= hidden_size
    loss=QuantileLoss(),
    optimizer="Ranger"
    # reduce learning rate if no improvement in validation loss after x epochs
    # reduce_on_plateau_patience=1000,
)
print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

Global seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Number of parameters in network: 6.2k


In [161]:
# find optimal learning rate
from lightning.pytorch.tuner import Tuner

res = Tuner(trainer).lr_find(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
    max_lr=10.0,
    min_lr=1e-6,
)

print(f"suggested learning rate: {res.suggestion()}")
fig = res.plot(show=True, suggest=True)
fig.show()

TypeError: `model` must be a `LightningModule` or `torch._dynamo.OptimizedModule`, got `TemporalFusionTransformer`

In [155]:
tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=LEARNING_RATE,
    hidden_size=16,  # biggest influence network size
    attention_head_size=1,
    dropout=0.1,
    hidden_continuous_size=8,
    output_size=7,  # QuantileLoss has 7 quantiles by default
    loss=QuantileLoss(),
    # reduce learning rate if no improvement in validation loss after x epochs
    reduce_on_plateau_patience=4,
)

print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

# t

Number of parameters in network: 18.6k


In [159]:
# calculate baseline mean absolute error, i.e. predict next value as the last available value from the history
baseline_predictions = Baseline().predict(val_dataloader, return_y=True)
MAE()(baseline_predictions.output, baseline_predictions.y)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Missing logger folder: /home/chris/dev/jhu/605.742_deep_neural_networks/final_project/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


tensor(6.0678, device='cuda:0')

# Generating Predictions

Only for Kaggle:
When uploading notebook, make sure to add the Optiver data on the right side of the screen.

In [ ]:
if kaggle_environment:
    import optiver2023
    env = optiver2023.make_env()
    iter_test = env.iter_test()

In [ ]:
if kaggle_environment:
    # To count how many time the "for loop" runs.
    counter = 0

    # init 3 empty lists
    test_ls, revealed_targets_ls, sample_prediction_ls = [], [], []

    for (test, revealed_targets, sample_prediction) in iter_test:
        # Append the dataframe that API return into the list.
        test_ls.append(test.copy())
        revealed_targets_ls.append(revealed_targets.copy())
        sample_prediction_ls.append(sample_prediction.copy())

        # Generate features
        test = generate_features(test)

        # Writes our predictions 
        sample_prediction["target"] = model.predict(test)
        
        # This line submit our predictions.
        env.predict(sample_prediction)
        counter += 1

    print('\n', '=' * 50, sep="")
    print(f"counter: {counter}")